# BERT with TF-IDF features

## Installation

In [1]:
# !pip install --upgrade pip # upgrade pip

In [2]:
# !pip install -U scikit-learn
# !pip install -U transformers
# !pip install -U datasets
# !pip install ipywidgets

For tqdm progress bars (on a terminal):
1. `conda install -c conda-forge nodejs`
2. `jupyter labextension install @jupyter-widgets/jupyterlab-manager`
3. `jupyter nbextension enable --py widgetsnbextension`
4. `jupyter lab clean`
5. Refresh web page...

In [3]:
# # Check versions
# import sklearn
# import transformers
# import datasets
# 
# print("Current versions:")
# print(sklearn.__version__)
# print(datasets.__version__)
# print(transformers.__version__)

## Libraries

In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath(".."))

import time
import random
import pickle

import numpy as np
import torch

from sklearn.linear_model import RidgeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

import src.model as mod
from src.train import *

In [5]:
%load_ext autoreload
%autoreload 2

## Device and seeds

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# seeds (torch generator seed missing?)
seed = 1979
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

## Parameters

In [7]:
# Dataset folder & device
dataset_name = 'imdb' # 'trec'
sort = True          # True

# Model 
model_name = 'bert-base-uncased'
tfidf_dim = 3000
batch_size = 128
pooling = 'mean'      # 'mean', 'mean_std', cls', 'mean_cls', 'mean_std_cls'.
mode = 'default'      # 'default', 'bert_only', 'tfidf_only'

# Learning algo
alpha = 10

# results
results_folder = "/raid/home/jeremiec/Data/TextClassification"
results_file = os.path.join(results_folder, dataset_name) + '.pkl'
cache_dir = os.path.join(results_folder, 'cache_dir_' + dataset_name + '/')

if os.path.exists(cache_dir):
    os.system("rm -rf " + cache_dir)

In [8]:
# x_tmp = torch.rand(size=(100000, 73500), device=device)

## Load and tokenize data

**Overview of the datasets**
1. Sentiment analysis
    - ``IMDB``
    - ``Yelp. P``
    - ``Yelp. F``
2. Question classification
    - ``TREC``
    - ``Yahoo! Answers``
3. Topic detection
    - ``AG News``
    - ``DBPedia``

In [9]:
# # Sentiment
# # IMDB
# dataset, tokenizer, model_name = load_and_tokenize_dataset('imdb', model_name=model_name, cache_dir=cache_dir)

# # Yelp P. XXX CUDA OoM
# dataset, tokenizer, model_name = load_and_tokenize_dataset('yelp_polarity', model_name=model_name, cache_dir=cache_dir)

# # Yelp F. XXX CUDA OoM
# dataset, tokenizer, model_name = load_and_tokenize_dataset('yelp_review_full', model_name=model_name, cache_dir=cache_dir)


# # Question
# # TREC
# dataset, tokenizer, model_name = load_and_tokenize_dataset('trec', model_name=model_name, cache_dir=cache_dir)

# # Yahoo! Answers # XXX
# dataset, tokenizer, model_name = load_and_tokenize_dataset('yahoo_answers_topics', model_name=model_name, cache_dir=cache_dir)


# # Topic
# # AG NEWS
# dataset, tokenizer, model_name = load_and_tokenize_dataset('ag_news', model_name=model_name, cache_dir=cache_dir)

# # DBPedia # XXX
# dataset, tokenizer, model_name = load_and_tokenize_dataset('dbpedia_14', model_name=model_name, cache_dir=cache_dir)



dataset, tokenizer, model_name = load_and_tokenize_dataset(dataset_name, 
                                                           model_name=model_name, 
                                                           sort=sort,
                                                           cache_dir=cache_dir)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /raid/home/jeremiec/Data/TextClassification/cache_dir_imdb/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'length'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'length'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'length'],
        num_rows: 50000
    })
})

## Model

In [12]:
t0 = time.time()

dataset = mod.get_tfidf_features(dataset, dim=tfidf_dim)

t1 = time.time()

tfidf_time = t1 - t0
print(f"Features computed in {tfidf_time} sec.")

Features computed in 33.72184348106384 sec.


In [13]:
model = mod.BertTFIDF(model_name=model_name, pooling=pooling, mode=mode, device=device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


## Training loop

In [14]:
# Processing data
t0 = time.time()

X_train, y_train = process_dataset(dataset['train'], model, tokenizer, device, batch_size)

t1 = time.time()

training_time = t1 - t0

  0%|          | 0/196 [00:00<?, ?it/s]

In [15]:
# Fitting the model
# Note: if different alpha are tested, insert a loop here
t0 = time.time()

learning_algo = RidgeClassifier(alpha=alpha)
learning_algo.fit(X_train, y_train)

t1 = time.time()

fitting_time = t1 - t0

In [16]:
# # OLD VERSION

# t0 = time.time()

# learning_algo = train_learning_algo(learning_algo, dataset, model, tokenizer, 
#                                     device, batch_size)

# t1 = time.time()

# training_time = t1 - t0
# print(print(f"Model trained in {training_time} sec."))

## Results

In [17]:
y_test, y_test_preds = predict(learning_algo, dataset, model, tokenizer, 
                               device, batch_size)

  0%|          | 0/196 [00:00<?, ?it/s]

In [18]:
# Results
test_results = classification_report(y_test, y_test_preds, digits=4, output_dict=True)
print(classification_report(y_test, y_test_preds, digits=4))

              precision    recall  f1-score   support

         0.0     0.9446    0.9582    0.9513     12500
         1.0     0.9576    0.9438    0.9506     12500

    accuracy                         0.9510     25000
   macro avg     0.9511    0.9510    0.9510     25000
weighted avg     0.9511    0.9510    0.9510     25000



In [19]:
test_results

{'0.0': {'precision': 0.9446328574808739,
  'recall': 0.95816,
  'f1-score': 0.9513483458437587,
  'support': 12500},
 '1.0': {'precision': 0.9575521467413359,
  'recall': 0.94384,
  'f1-score': 0.9506466298698683,
  'support': 12500},
 'accuracy': 0.951,
 'macro avg': {'precision': 0.951092502111105,
  'recall': 0.9510000000000001,
  'f1-score': 0.9509974878568135,
  'support': 25000},
 'weighted avg': {'precision': 0.9510925021111049,
  'recall': 0.951,
  'f1-score': 0.9509974878568135,
  'support': 25000}}

In [20]:
print(f"Total training time: {tfidf_time + training_time + fitting_time} sec.")

Total training time: 163.60133409500122 sec.


In [21]:
# del dataset, tokenizer, model, learning_algo
torch.cuda.empty_cache()

In [22]:
# save results

if os.path.exists(results_file):
    with open(results_file, 'rb') as fh:
        results_d = pickle.load(fh)
else:
    results_d = {}

    
key = (pooling, mode, tfidf_dim, alpha, batch_size)
results_d[key] = (test_results, 
                  tfidf_time + training_time + fitting_time, 
                  "pooling - mode - tfidf_dim - alpha - batch_size")

with open(results_file, 'wb') as fh:
    pickle.dump(results_d, fh)

In [23]:
results_d

{('mean',
  'default',
  3000,
  10,
  32): ({'0.0': {'precision': 0.900877893056664,
    'recall': 0.90304,
    'f1-score': 0.9019576508190172,
    'support': 12500},
   '1.0': {'precision': 0.9028067361668003,
    'recall': 0.90064,
    'f1-score': 0.9017220664797757,
    'support': 12500},
   'accuracy': 0.90184,
   'macro avg': {'precision': 0.9018423146117321,
    'recall': 0.90184,
    'f1-score': 0.9018398586493965,
    'support': 25000},
   'weighted avg': {'precision': 0.9018423146117321,
    'recall': 0.90184,
    'f1-score': 0.9018398586493965,
    'support': 25000}}, 291.1757595539093, 'pooling - mode - tfidf_dim - alpha - batch_size'),
 ('mean',
  'default',
  3000,
  10,
  128): ({'0.0': {'precision': 0.9446328574808739,
    'recall': 0.95816,
    'f1-score': 0.9513483458437587,
    'support': 12500},
   '1.0': {'precision': 0.9575521467413359,
    'recall': 0.94384,
    'f1-score': 0.9506466298698683,
    'support': 12500},
   'accuracy': 0.951,
   'macro avg': {'precisi

**Remarks**

- We chose to implement the **standard mean:** ``torch.mean(batch, dim=1)``: <br>
    Instead of a custom mean where we sum the embedded tokens and divide by the sentence length, we implement a standard mean of the batch. After some experiments, this seems to work better.
- Similarly, we chose to implement the **standard std:** ``torch.std(batch, dim=1)``: <br>
    Instead of a custom std involving the sentence length, we implement a standard std of the batch. After some experiments, this seems to work better.
- The **sorting by length** process infuences the results: <br>
For some datasets, the results are better when the data are sorted by length, while for others, the opposite holds true. This phenomenon is probably due to the mean operation applied to the batch, which would yield very different results depending on whether the batch is sorted or not.
- The **batch size** singificantly influences the results: <br>
    Large batches include more padding than small batches. These padded tokens are actually involved in the computation of the mean, and thus influences it. For this reason, the batch size influences the results. The batch size will be a hyperparameter of our model.